In [1]:
import dask
import numpy as np
import xarray as xr
import glob
import matplotlib.pyplot as plt
import time
import datetime
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import warnings

In [2]:
warnings.filterwarnings('ignore')
now = datetime.datetime.now()
now_string = str(now.strftime("%Y-%m-%d_%A_%H%M%S"))
now_string

'2019-07-11_Thursday_130815'

In [3]:
cluster = SLURMCluster(cores=16, memory='20GB', project='pi_jianwu', queue='high_mem', job_extra=['--qos=medium+','--exclusive'])

In [4]:
cluster.scale(8)

In [5]:
!squeue -u savio1

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1080298  high_mem dask-wor   savio1 CF       0:01      1 cnode033
           1080299  high_mem dask-wor   savio1 CF       0:01      1 cnode034
           1080300  high_mem dask-wor   savio1 CF       0:01      1 cnode043
           1080301  high_mem dask-wor   savio1 CF       0:01      1 cnode044
           1080302  high_mem dask-wor   savio1 CF       0:01      1 cnode002
           1080303  high_mem dask-wor   savio1 CF       0:01      1 cnode003
           1080304  high_mem dask-wor   savio1 PD       0:00      1 (QOSMaxCpuPerUserLimit)
           1080305  high_mem dask-wor   savio1 PD       0:00      1 (QOSMaxCpuPerUserLimit)
           1080297  high_mem   tunnel   savio1  R       6:15      4 cnode[039-042]


In [ ]:
client = Client()

In [7]:
cluster.dashboard_link

'http://10.2.1.39:8787/status'

In [8]:
cluster.running_jobs

OrderedDict()

In [9]:
cluster.pending_jobs

OrderedDict([('1080298', {}),
             ('1080299', {}),
             ('1080300', {}),
             ('1080301', {}),
             ('1080302', {}),
             ('1080303', {}),
             ('1080304', {}),
             ('1080305', {})])

In [10]:
cluster.scheduler_address

'tcp://10.2.1.39:34010'

In [18]:
client = Client()

distributed.nanny - WARNING - Worker process still alive after 47 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 47 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 47 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 47 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 47 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 47 seconds, killing
distributed.nanny - WARNING - Worker process 374139 was killed by signal 15
distributed.nanny - WARNING - Worker process 374142 was killed by signal 15
distributed.nanny - WARNING - Worker process 374145 was killed by signal 15
distributed.nanny - WARNING - Worker process 374148 was killed by signal 15
distributed.nanny - WARNING - Worker process 374155 was killed by signal 15
distributed.nanny - WARNING - Worker process 374151 was killed by signal 15
distributed.nanny - WARNING - Worker process s

TimeoutError: Worker failed to start

In [19]:
print(client)

<Client: scheduler='tcp://10.2.1.39:34010' processes=0 cores=0>


In [20]:
cluster.dashboard_link

'http://10.2.1.39:8787/status'

In [22]:
cluster.start_workers

<bound method JobQueueCluster.start_workers of SLURMCluster(cores=0, memory=0 B, workers=0/8, jobs=0/8)>

In [23]:
cluster.scale(8)

In [24]:
cluster.close()

Exception ignored in: <generator object Scheduler.add_client at 0x2aaae7a84de0>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object Scheduler.add_client at 0x2aaae7b31f48>
RuntimeError: generator ignored GeneratorExit


In [12]:
t0 = time.time()
total_pix = np.zeros((180, 360))
cloud_pix = np.zeros((180, 360))

In [13]:
def ingest_data(M03_dir, M06_dir):
    M03_files = sorted(glob.glob(M03_dir + "MYD03.A2008*.hdf"))
    M06_files = sorted(glob.glob(M06_dir + "MYD06_L2.A2008*.hdf"))
    for M03, M06 in zip (M03_files, M06_files):
        d06 = xr.open_mfdataset(M06[:], parallel=True)['Cloud_Mask_1km'][:,:,:].values
        d06CM = d06[::3,::3,0]
        ds06_decoded = (np.array(d06CM, dtype = "byte") & 0b00000110) >> 1
        d03_lat = xr.open_mfdataset(M03[:], drop_variables = "Scan Type", parallel=True)['Latitude'][:,:].values
        d03_lon = xr.open_mfdataset(M03[:], drop_variables = "Scan Type", parallel=True)['Longitude'][:,:].values

        lat = d03_lat[::3,::3]
        lon = d03_lon[::3,::3]

        l_index = (lat + 89.5).astype(int).reshape(lat.shape[0]*lat.shape[1])
        lat_index = np.where(l_index > -1, l_index, 0)
        ll_index = (lon + 179.5).astype(int).reshape(lon.shape[0]*lon.shape[1])
        lon_index = np.where(ll_index > -1, ll_index, 0)
        for i, j in zip(lat_index, lon_index):
            total_pix[i,j] += 1

        indicies = np.nonzero(ds06_decoded <= 0)
        row_i = indicies[0]
        column_i = indicies[1]
        cloud_lon = [lon_index.reshape(ds06_decoded.shape[0],ds06_decoded.shape[1])[i,j] for i, j in zip(row_i, column_i)]
        cloud_lat = [lat_index.reshape(ds06_decoded.shape[0],ds06_decoded.shape[1])[i,j] for i, j in zip(row_i, column_i)]

        for x, y in zip(cloud_lat, cloud_lon):
            cloud_pix[int(x),int(y)] += 1
            
    return cloud_pix, total_pix

In [ ]:
t0 = time.time()
import dask.multiprocessing
dask.config.set(num_workers=8)
M03_dir = "/home/savio1/cybertrn_common/Data/Satellite_Observations/MODIS/MYD03/"
M06_dir = "/home/savio1/cybertrn_common/Data/Satellite_Observations/MODIS/MYD06_L2/"
future1 = client.submit(ingest_data,M03_dir,M06_dir)
result1 = client.gather(future1)

In [ ]:
future1.result()

In [ ]:
cf1 = future1.result()[0]/future1.result()[1]

In [ ]:
plt.figure(figsize=(14,7))
plt.contourf(range(-180,180), range(-90,90), cf1, 100, cmap = "jet")
plt.xlabel("Longitude", fontsize = 14)
plt.ylabel("Latitude", fontsize = 14)
plt.title("Level 3 Cloud Fraction Aggregation For One Month %s" %now_string, fontsize = 16)
plt.colorbar()
plt.savefig("/umbc/xfs1/jianwu/common/MODIS_Aggregation/savioexe/test/8/%s.png" %now_string)

In [ ]:
cf2 = xr.DataArray(cf1)
cf2.to_netcdf("/umbc/xfs1/jianwu/common/MODIS_Aggregation/savioexe/test/8/%s.hdf" %now_string)

In [ ]:
t1 = time.time()
total = t1-t0
print(total)

In [ ]:
print(total/3600,"hours")

In [ ]:
cluster.pending_jobs

In [ ]:
cluster.close()

In [ ]:
client.close()

In [ ]:
!squeue -u savio1